## Make Training Data for Species Modeling from NEON TOS Vegetation Structure Data

This notebook walks through generating a training dataset (namely obtaining the tree species, family, and location) from NEON Terrestrial Observation System (TOS) Vegetation Structure data (DP1.10098.001). A random forest machine learning model can be applied, using this training data, to predict the family based off of the hyperspectral signature from the airborne hyperspectral data.

### Resources

The lesson "Compare tree height measured from the ground to a Lidar-based Canopy Height Model" may be a useful resource
https://www.neonscience.org/resources/learning-hub/tutorials/tree-heights-veg-structure-chm

Individual canopy tree species maps for the National Ecological Observatory Network
https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.3002700

Science Seminar
https://www.neonscience.org/get-involved/events/science-seminar-harnessing-neon-enable-future-forest-remote-sensing

Refer to the <a href="https://data.neonscience.org/api/v0/documents/NEON_vegStructure_userGuide_vE?inline=true" target=_blank> vegetation structure user guide</a> for more details on this data product, and to better understand the data quality flags, the sampling design, and other important information that will determine how you carry out your analysis.

**Disclaimer**: this notebook is intended to provide an example of how to procure training data and conduct some exploratory analysis, it does not necessarily represent the best method or all recommended pre-processing steps. A number of simplifications / assumptions are made along the way, including:

...

In [1]:
import os
# import georasters as gr
from math import floor, ceil
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
import neonutilities as nu
import numpy as np
from osgeo import gdal
import pandas as pd
import rasterio as rio
from rasterio.merge import merge
from rasterio.plot import show
import requests
import seaborn as sns

## Download and Explore Vegetation Structure Data (DP1.10098.001)

In this section we’ll download the vegetation structure data, find the locations of the mapped trees, and join to the species and DBH (diameter at breast height) data.

Download the vegetation structure data using the `load_by_product` function in the `neonutilities` package (imported as `nu`). Inputs to the function can be shown by typing `help(load_by_product)`.
  
Refer to th e<a href=https://www.neonscience.org/sites/default/files/cheat-sheet-neonUtilities.pdf target=_blank> R neonutilities cheat sheet</a>  or the neonUtilities package for more details and the complete index of possible function inputs. The cheat sheet is focused on the R package, but nearly all the inputs are the same

Note that in this example, we will pull in all the woody vegetation data (collected over all years), but if you are trying to model heights measured in a single year, you can select just that year by specifying the `startdate` and `enddate`, or later filtering out the vegetation data by the eventID. See the <a href=https://github.com/NEONScience/NEON-Data-Skills/blob/main/tutorials/R/AOP/Lidar/lidar-topography/veg_structure_and_chm/veg_structure_and_chm-content.html target=_blank> vegetation structure and chm lesson</a> for more details on how to filter by eventID.

In [2]:
veg_dict = nu.load_by_product(dpid="DP1.10098.001", 
                              site="SERC", 
                              package="basic", 
                              release="RELEASE-2025",
                              check_size=False)

Finding available files
100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:23<00:00,  1.01s/it]
C:\Users\bhass\.conda\envs\lpdaac_vitals\lib\site-packages\neonutilities\unzip_and_stack.py:140: UserWarning: Filepaths on Windows are limited to 260 characters. Attempting to extract a filepath that is > 260 characters long. Move your working or savepath directory closer to the root directory or enable long path support in Windows.
  warnings.warn(
Stacking data files
100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.70it/s]


Get a list of the points

In [3]:
veg_map_all = veg_dict["vst_mappingandtagging"]
veg_map = veg_map_all.loc[veg_map_all["pointID"] != ""]
veg_map = veg_map.reindex()
veg_map["points"] = veg_map["namedLocation"] + "." + veg_map["pointID"]
veg_points = list(set(list(veg_map["points"])))

Look at the unique `eventID`s. 

In [4]:
veg_map_all.eventID.unique()

array(['vst_SERC_2015', 'vst_SERC_2016', 'vst_SERC_2017', 'vst_SERC_2018',
       'vst_SERC_2019', 'vst_SERC_2020', 'vst_SERC_2021', 'vst_SERC_2022',
       'vst_SERC_2023'], dtype=object)

Get the number of records for each eventID:

In [5]:
# Group by 'eventID' and get the count
eventID_counts = veg_map_all[['individualID','eventID']].groupby(['eventID']).count()
print("\nCounts of each eventID:\n", eventID_counts)


Counts of each eventID:
                individualID
eventID                    
vst_SERC_2015          1890
vst_SERC_2016          1330
vst_SERC_2017            96
vst_SERC_2018           127
vst_SERC_2019           254
vst_SERC_2020            22
vst_SERC_2021            54
vst_SERC_2022           494
vst_SERC_2023            40


It looks like most of the trees were mapped in 2015 and 2016, which was when the SERC plots were first established. You could look at data only from one year, and compare to AOP data from the same year (see below to determine which dates of AOP data are available), or if you are not too worried about matching measurements to remote sensing data collected in the same year, you could use all years. We'll do that in this example, but you have some options here.

## Determine the geographic location of the surveyed vegetation

Loop through all of the points in `veg_points` to determine the easting and norhting from the locations API

In [6]:
easting = []
northing = []
coord_uncertainty = []
elev_uncertainty = []
for i in veg_points:
    vres = requests.get("https://data.neonscience.org/api/v0/locations/"+i)
    vres_json = vres.json()
    easting.append(vres_json["data"]["locationUtmEasting"])
    northing.append(vres_json["data"]["locationUtmNorthing"])
    props = pd.DataFrame.from_dict(vres_json["data"]["locationProperties"])
    cu = props.loc[props["locationPropertyName"]=="Value for Coordinate uncertainty"]["locationPropertyValue"]
    coord_uncertainty.append(cu[cu.index[0]])
    eu = props.loc[props["locationPropertyName"]=="Value for Elevation uncertainty"]["locationPropertyValue"]
    elev_uncertainty.append(eu[eu.index[0]])

pt_dict = dict(points=veg_points, 
               easting=easting,
               northing=northing,
               coordinateUncertainty=coord_uncertainty,
               elevationUncertainty=elev_uncertainty)

pt_df = pd.DataFrame.from_dict(pt_dict)
pt_df.set_index("points", inplace=True)

veg_map = veg_map.join(pt_df, 
                     on="points", 
                     how="inner")

Next, use the stemDistance and stemAzimuth data to calculate the precise locations of individuals, relative to the reference locations.

- $Easting = easting.pointID + stemDistance*sin(\theta)$
- $Northing = northing.pointID + stemDistance*cos(\theta)$
- $\theta = stemAzimuth*\pi/180$

Also adjust the coordinate and elevation uncertainties.

In [7]:
veg_map["adjEasting"] = (veg_map["easting"]
                        + veg_map["stemDistance"]
                        * np.sin(veg_map["stemAzimuth"]
                                   * np.pi / 180))

veg_map["adjNorthing"] = (veg_map["northing"]
                        + veg_map["stemDistance"]
                        * np.cos(veg_map["stemAzimuth"]
                                   * np.pi / 180))

veg_map["adjCoordinateUncertainty"] = veg_map["coordinateUncertainty"] + 0.6

veg_map["adjElevationUncertainty"] = veg_map["elevationUncertainty"] + 1

Look at the columns to see all the information contained in this dataset.

In [8]:
veg_map.columns

Index(['uid', 'namedLocation', 'date', 'eventID', 'domainID', 'siteID',
       'plotID', 'pointID', 'stemDistance', 'stemAzimuth', 'recordType',
       'individualID', 'supportingStemIndividualID', 'previouslyTaggedAs',
       'otherTagID', 'otherTagOrg', 'samplingProtocolVersion',
       'identificationHistoryID', 'taxonID', 'scientificName', 'genus',
       'family', 'taxonRank', 'identificationReferences', 'morphospeciesID',
       'morphospeciesIDRemarks', 'identificationQualifier', 'remarks',
       'measuredBy', 'recordedBy', 'dataQF', 'publicationDate', 'release',
       'points', 'easting', 'northing', 'coordinateUncertainty',
       'elevationUncertainty', 'adjEasting', 'adjNorthing',
       'adjCoordinateUncertainty', 'adjElevationUncertainty'],
      dtype='object')

### Combine location with tree traits

Now we have the mapped locations of individuals in the `vst_mappingandtagging` table, and the annual measurements of tree dimensions such as height and diameter in the vst_apparentindividual table. To bring these measurements together, join the two tables. Refer to the [Quick Start Guide for Vegetation structure]() for more information about the data tables and the joining instructions.

In [9]:
veg_dict["vst_apparentindividual"].set_index("individualID", inplace=True)
veg = veg_map.join(veg_dict["vst_apparentindividual"],
                   on="individualID",
                   how="inner",
                   lsuffix="_MAT",
                   rsuffix="_AI")

In [10]:
# show all the columns in the joined veg dataset
veg.columns

Index(['uid_MAT', 'namedLocation_MAT', 'date_MAT', 'eventID_MAT',
       'domainID_MAT', 'siteID_MAT', 'plotID_MAT', 'pointID', 'stemDistance',
       'stemAzimuth', 'recordType', 'individualID',
       'supportingStemIndividualID', 'previouslyTaggedAs', 'otherTagID',
       'otherTagOrg', 'samplingProtocolVersion', 'identificationHistoryID',
       'taxonID', 'scientificName', 'genus', 'family', 'taxonRank',
       'identificationReferences', 'morphospeciesID', 'morphospeciesIDRemarks',
       'identificationQualifier', 'remarks_MAT', 'measuredBy_MAT',
       'recordedBy_MAT', 'dataQF_MAT', 'publicationDate_MAT', 'release_MAT',
       'points', 'easting', 'northing', 'coordinateUncertainty',
       'elevationUncertainty', 'adjEasting', 'adjNorthing',
       'adjCoordinateUncertainty', 'adjElevationUncertainty', 'uid_AI',
       'namedLocation_AI', 'date_AI', 'eventID_AI', 'domainID_AI', 'siteID_AI',
       'plotID_AI', 'subplotID', 'tempStemID', 'tagStatus', 'growthForm',
       'plan

In [11]:
# pull out a subset of the columns that may be relevant
veg[['date_AI','individualID','scientificName','taxonID','family','growthForm','plantStatus','plotID_AI','pointID','stemDiameter','adjEasting','adjNorthing']].head(5)

,date_AI,individualID,scientificName,taxonID,family,growthForm,plantStatus,plotID_AI,pointID,stemDiameter,adjEasting,adjNorthing
1,2015-09-28,NEON.PLA.D02.SERC.08038,Carya glabra (Mill.) Sweet,CAGL8,Juglandaceae,single bole tree,Live,SERC_045,43,46.4,364809.083993,4.304727e+06
1,2016-10-11,NEON.PLA.D02.SERC.08038,Carya glabra (Mill.) Sweet,CAGL8,Juglandaceae,single bole tree,Live,SERC_045,43,47.1,364809.083993,4.304727e+06
1,2017-11-16,NEON.PLA.D02.SERC.08038,Carya glabra (Mill.) Sweet,CAGL8,Juglandaceae,single bole tree,Live,SERC_045,43,47.4,364809.083993,4.304727e+06
1,2018-10-02,NEON.PLA.D02.SERC.08038,Carya glabra (Mill.) Sweet,CAGL8,Juglandaceae,single bole tree,Live,SERC_045,43,47.9,364809.083993,4.304727e+06
1,2019-11-07,NEON.PLA.D02.SERC.08038,Carya glabra (Mill.) Sweet,CAGL8,Juglandaceae,single bole tree,Live,SERC_045,43,47.9,364809.083993,4.304727e+06


## Remove duplicate records

We will select only the latest recorded data so as to use the most recent data and remove any duplicated data points.

In [12]:
# Convert 'date_AI' to datetime if it's not already
veg.loc[:, 'date_AI'] = pd.to_datetime(veg['date_AI'])

# Sort the DataFrame by 'individualID' and 'date_AI' in descending order
veg_date_sorted = veg.sort_values(by=['individualID', 'date_AI'], ascending=[True, False])

# Drop duplicates, keeping the first occurrence (which is the latest, or most recent, due to sorting)
veg_latest = veg_date_sorted.drop_duplicates(subset='individualID', keep='first').copy()

# Display the DataFrame with only the latest entries for each individualID
print(len(veg_latest))

# Display a subset of the columns
veg_latest[['date_AI','individualID','scientificName','taxonID','family','growthForm','plantStatus','plotID_AI','pointID','stemDiameter','adjEasting','adjNorthing']].head(5)

1185


,date_AI,individualID,scientificName,taxonID,family,growthForm,plantStatus,plotID_AI,pointID,stemDiameter,adjEasting,adjNorthing
805,2022-12-05 00:00:00,NEON.PLA.D02.SERC.00017,Acer rubrum L.,ACRU,Aceraceae,single bole tree,Live,SERC_054,39,27.5,365000.851457,4.305652e+06
823,2022-12-05 00:00:00,NEON.PLA.D02.SERC.00029,Fagus grandifolia Ehrh.,FAGR,Fagaceae,single bole tree,Live,SERC_054,57,34.2,365008.826768,4.305655e+06
819,2022-12-05 00:00:00,NEON.PLA.D02.SERC.00032,Fagus grandifolia Ehrh.,FAGR,Fagaceae,single bole tree,Live,SERC_054,39,32.0,365002.039970,4.305661e+06
3865,2022-12-05 00:00:00,NEON.PLA.D02.SERC.00039,Fagus grandifolia Ehrh.,FAGR,Fagaceae,single bole tree,Live,SERC_054,57,10.7,364993.798628,4.305656e+06
3136,2022-12-05 00:00:00,NEON.PLA.D02.SERC.00041,Liquidambar styraciflua L.,LIST2,Hamamelidaceae,small tree,Standing dead,SERC_054,57,9.7,364990.841914,4.305660e+06


Display the # of unique dates

In [13]:
veg_dates = veg_latest['date_AI'].unique()
veg_dates.sort()

Now create a new dataframe containing only the veg that are within a single tile. For this you will need to know the bounds (minimum and maximum UTM easting and northing) of the area you are sampling. 

In [14]:
veg_tower_tile = veg_latest[(veg_latest['adjEasting'].between(364000, 365000)) & (veg_latest['adjNorthing'].between(4305000, 4306000))]

In [15]:
veg_tower_tile.head()

,uid_MAT,namedLocation_MAT,date_MAT,eventID_MAT,domainID_MAT,siteID_MAT,plotID_MAT,pointID,stemDistance,stemAzimuth,...,dendrometerGap,dendrometerCondition,bandStemDiameter,remarks_AI,recordedBy_AI,measuredBy_AI,dataEntryRecordID,dataQF_AI,publicationDate_AI,release_AI
3865,eedf11b5-8630-4360-8b43-fe0b684dd765,SERC_054.basePlot.vst,2022-12-05,vst_SERC_2022,D02,SERC,SERC_054,57,6.3,134.4,...,NaN,,NaN,,0000-0001-6848-3983,0000-0003-1337-5330,03d2cf4c-6408-4666-8a4e-5e513b68ea1a,,20241118T020758Z,RELEASE-2025
3136,acd34e71-b551-4ccb-91c8-b894c7ef0641,SERC_054.basePlot.vst,2016-10-13,vst_SERC_2016,D02,SERC,SERC_054,57,1.7,114.7,...,NaN,,NaN,Unknown cause of death,0000-0001-6848-3983,0000-0003-1337-5330,a6b2f48e-e4d7-43c6-9c5f-5dd260f53171,,20241118T020758Z,RELEASE-2025
800,a17acc40-b849-4347-bedd-44bd9556b247,SERC_054.basePlot.vst,2015-05-08,vst_SERC_2015,D02,SERC,SERC_054,57,5.8,154.5,...,NaN,,NaN,Unknown cause of death,0000-0001-6848-3983,0000-0003-1337-5330,4886642a-7e22-4148-ba9c-fb4a5d1ed673,,20241118T020758Z,RELEASE-2025
911,2904a076-87d7-452e-80da-f47ff71c7617,SERC_054.basePlot.vst,2015-05-08,vst_SERC_2015,D02,SERC,SERC_054,39,8.2,9.6,...,NaN,,NaN,,0000-0001-6848-3983,0000-0003-1337-5330,23e411a0-2d2f-4b8a-96cb-1865114a8149,,20241118T020758Z,RELEASE-2025
1762,2d46184e-7643-4bab-b5ad-ef2be42a9b99,SERC_047.basePlot.vst,2015-09-29,vst_SERC_2015,D02,SERC,SERC_047,21,13.9,78.0,...,NaN,,NaN,,0000-0002-3485-3744,0000-0001-6848-3983,99869248-b367-4001-ab99-9ba4715ca5ac,,20241119T221759Z,RELEASE-2025


In [16]:
# look at the unique Taxon IDs
veg_tower_tile.taxonID.unique()

array(['FAGR', 'LIST2', 'QUFA', 'LITU', 'ACRU', 'CACA18', 'NYSY', 'ULMUS',
       'CAGL8', 'QURU', 'QUAL', 'CATO6', 'PINUS', 'QUERC', 'COFL2',
       'PRAV', 'QUVE'], dtype=object)

In [17]:
veg_tower_tile_short = veg_tower_tile[['date_AI','individualID','scientificName','taxonID','family','growthForm','plantStatus','plotID_AI','pointID','stemDiameter','adjEasting','adjNorthing']]
veg_tower_tile_short

,date_AI,individualID,scientificName,taxonID,family,growthForm,plantStatus,plotID_AI,pointID,stemDiameter,adjEasting,adjNorthing
3865,2022-12-05 00:00:00,NEON.PLA.D02.SERC.00039,Fagus grandifolia Ehrh.,FAGR,Fagaceae,single bole tree,Live,SERC_054,57,10.7,364993.798628,4.305656e+06
3136,2022-12-05 00:00:00,NEON.PLA.D02.SERC.00041,Liquidambar styraciflua L.,LIST2,Hamamelidaceae,small tree,Standing dead,SERC_054,57,9.7,364990.841914,4.305660e+06
800,2022-12-05 00:00:00,NEON.PLA.D02.SERC.00043,Quercus falcata Michx.,QUFA,Fagaceae,single bole tree,"Dead, broken bole",SERC_054,57,37.1,364991.794414,4.305655e+06
911,2022-12-05 00:00:00,NEON.PLA.D02.SERC.00062,Quercus falcata Michx.,QUFA,Fagaceae,single bole tree,Live,SERC_054,39,59.7,364990.424114,4.305650e+06
1762,2023-11-29 00:00:00,NEON.PLA.D02.SERC.00173,Fagus grandifolia Ehrh.,FAGR,Fagaceae,single bole tree,Live,SERC_047,21,22.5,364673.259742,4.305225e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
3787,2023-11-29 00:00:00,NEON.PLA.D02.SERC.14231,Fagus grandifolia Ehrh.,FAGR,Fagaceae,single bole tree,Live,SERC_047,23,10.8,364679.533579,4.305222e+06
4103,2023-01-16 00:00:00,NEON.PLA.D02.SERC.14548,Fagus grandifolia Ehrh.,FAGR,Fagaceae,single bole tree,Live,SERC_057,41,11.1,364455.772024,4.305415e+06
4096,2023-01-16 00:00:00,NEON.PLA.D02.SERC.14563,Quercus alba L.,QUAL,Fagaceae,single bole tree,Live,SERC_057,43,10.1,364470.101669,4.305412e+06
3818,2022-11-15 00:00:00,NEON.PLA.D02.SERC.14709,Fagus grandifolia Ehrh.,FAGR,Fagaceae,single bole tree,Live,SERC_052,41,10.4,364577.886559,4.305883e+06


In [18]:
len(veg_tower_tile_short)

207

Our final dataset has 207 records. Let's look at th eunique taxon and families represented as follows:

In [19]:
veg_tower_tile.taxonID.unique()

array(['FAGR', 'LIST2', 'QUFA', 'LITU', 'ACRU', 'CACA18', 'NYSY', 'ULMUS',
       'CAGL8', 'QURU', 'QUAL', 'CATO6', 'PINUS', 'QUERC', 'COFL2',
       'PRAV', 'QUVE'], dtype=object)

In [20]:
# display the taxonID counts, sorted descending
veg_tower_tile_taxon_counts = veg_tower_tile[['individualID','taxonID']].groupby(['taxonID']).count()
veg_tower_tile_taxon_counts.sort_values(by='individualID',ascending=False)

,individualID
taxonID,
FAGR,47
LITU,35
LIST2,29
ACRU,16
CAGL8,12
CACA18,11
QUAL,10
NYSY,10
CATO6,10


In [21]:
# display the family counts, sorted descending
veg_tower_tile_family_counts = veg_tower_tile[['individualID','family']].groupby(['family']).count()
veg_tower_tile_family_counts.sort_values(by='individualID',ascending=False)

,individualID
family,
Fagaceae,73
Magnoliaceae,35
Hamamelidaceae,29
Juglandaceae,22
Aceraceae,16
Cornaceae,13
Betulaceae,11
Ulmaceae,5
Pinaceae,2


It looks like there are a number of species mapped in this tower plot:

You can use the plants.usda.gov website to look up the species information. The top 5 most abundant mapped species are linked below.

- [FAGR](https://plants.usda.gov/plant-profile/FAGR): American Beech (Fagus grandifolia Ehrh.)
- [LITU](https://plants.usda.gov/plant-profile/LITU): Tuliptree (Liriodendron tulipifera L.)
- [LIST2](https://plants.usda.gov/plant-profile/LIST2): Sweetgum (Liquidambar styraciflua L.)
- [ACRU](https://plants.usda.gov/plant-profile/ACRU): Red Maple (Acer rubrum L.)
- [CAGL8](https://plants.usda.gov/plant-profile/CAGL8): Sweet pignut hickory (Carya glabra (Mill.))

When carrying out classification, the species that only have small representation (1-5 samples) may not be mapped accurately. The challenge of mapping rarer species due to insufficient training data is well known (and is sometimes called the 'long-tail' effect) e.g. We will end up removing these when we generate our model. 

Zhang, C., Chen, Y., Xu, B. et al. Improving prediction of rare species’ distribution from community data. Sci Rep 10, 12230 (2020). https://doi.org/10.1038/s41598-020-69157-x

Nonetheless, we have a fairly decent training dataset to work with. We can save the dataframe to a csv file called `serc_training_data.csv` as follows:

In [22]:
veg_tower_tile_short.to_csv(r'.\data\serc_training_data.csv',index=False)

## Recap

Great, we have now curated a training data set containing information about the tree family and species as well as it's geographic location in UTM x, y coordinates. We can now pair this dataset with remote sensing data and create a model to predict the tree's family based off airborne spectral data.